In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints

import nest_asyncio
nest_asyncio.apply()

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

sys.path.append("../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_jitted_funs
from python_helpers import helper_get_alpha

from python_helpers.data_figures import data_phase_equilibria_fluid_lr
from python_helpers.data_figures import data_critical_point_feanneos

In [2]:
##########################
# Folder to save results #
##########################

folder_to_save = '../computed_files'
os.makedirs(folder_to_save, exist_ok=True)

plot_figure = False # already checked that it converged nicely
save_files = True

In [3]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic = helper_jitted_funs(helmholtz_model, helmholtz_params)

In [4]:
lr_min = 7.
lr_max = 40.
lambda_a = 6.
inc0 = [0.3, 2.0]
n_crit = 100

# FE-ANN EoS
df_crit_feann = data_critical_point_feanneos(fun_dic, lr_min, lr_max, n_crit, lambda_a=lambda_a, inc0=inc0)
interp1d_crit = interp1d(df_crit_feann['lr'], df_crit_feann[['rhocad','Tcad']].T, kind='cubic')
df_crit_feann.head()

2025-05-13 09:17:43.072663: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


,lr,la,alpha,Tcad,rhocad,Pcad
0,7.000000,6.0,1.470949,2.062771,0.314607,0.219178
1,7.333333,6.0,1.391662,1.970107,0.315464,0.210149
2,7.666667,6.0,1.323481,1.888080,0.316516,0.202289
3,8.000000,6.0,1.264198,1.815308,0.317627,0.195407
4,8.333333,6.0,1.212155,1.750517,0.318735,0.189343


In [5]:
n = 100
lr_array = np.linspace(lr_min, lr_max, n)
rho_max_sle = 1.2
n_vle = 100
df_info_list = []
df_vle_list = []

for i in range(n):
    lambda_r = lr_array[i]
    initial_crit_point = interp1d_crit(lambda_r)
    phase_equilibria_lr = data_phase_equilibria_fluid_lr(fun_dic, lambda_r=lambda_r, initial_crit_point=initial_crit_point, rho_max_sle=rho_max_sle, n_vle=n_vle)
    df_info = phase_equilibria_lr['info']
    df_vle = phase_equilibria_lr['vle']
    df_vle['lambda_r'] = 1. * lambda_r
    df_vle['lambda_a'] = 6.
    df_vle['alpha'] = helper_get_alpha(df_vle['lambda_r'], df_vle['lambda_a'])
    df_vle['Tr_vle_model'] = df_vle['T_vle_model'].to_numpy() / df_info['Tcad_model'].to_numpy()
    df_info_list.append(df_info)
    df_vle_list.append(df_vle)
    if plot_figure:
        fig = plt.figure(figsize=(10, 3))
        ax1 = fig.add_subplot(131)
        ax2 = fig.add_subplot(132)
        ax3 = fig.add_subplot(133)

        ax1.plot(df_vle['rhov_vle_model'], df_vle['T_vle_model'], color='k')
        ax1.plot(df_vle['rhol_vle_model'], df_vle['T_vle_model'], color='k')

        ax2.semilogy(1/df_vle['T_vle_model'], df_vle['P_vle_model'], color='k')

        ax3.plot(df_vle['T_vle_model'], df_vle['Hvap_vle_model'], color='k')

        fig.suptitle(f'lambda_r = {lambda_r:.2f}')

    


2025-05-13 09:17:44.377285: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-05-13 09:17:44.482964: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-05-13 09:17:44.483907: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-05-13 09:17:44.507242: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


In [6]:
df_info_all = pd.concat(df_info_list).reset_index(drop=True)
df_vle_all = pd.concat(df_vle_list).reset_index(drop=True)
if save_files:
    filename = 'phase_equilibria_fluid.xlsx'
    file_to_save = os.path.join(folder_to_save, filename)
    writer = pd.ExcelWriter(file_to_save, engine='xlsxwriter')
    df_info_all.to_excel(writer, sheet_name='info', index=False)
    df_vle_all.to_excel(writer, sheet_name='vle', index=False)
    writer.close()
    print(f'File saved in {file_to_save}')

File saved in ../computed_files/phase_equilibria_fluid.xlsx
